In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

# Cargamos el dataset
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_2/DATOS_PREPROCESADOS_2.csv')

# Eliminar filas que contengan "-999" o -999 en cualquier columna
df = df[~df.isin(["-999", -999]).any(axis=1)]

# Eliminamos la columna identificadora
df.drop("gidenpac", axis=1, inplace=True)

# Lista de variables categóricas que se desean tratar como tal
cat_features = [
    'gsitalta', 'itipsexo', 'itipingr', 'ireingre', 'iotrocen', 'gdiagalt',
    'ds_izq_der', 'ds_turno', 'ds_dia_semana_llegada', 'ds_mes_llegada',
    'ds_centro_afueras', 'ds_alergia_medicamentosa', 'ds_alergia_alimenticia',
    'ds_otras_alergias', 'ds_ITU', 'ds_anemia',
    'ds_vitamina_d', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca',
    'ds_deterioro_cognitivo', 'ds_insuficiencia_renal', 'ds_HTA', 'ds_diabetes'
]

# Convertir las variables listadas a tipo 'category'
for col in cat_features:
    if col in df.columns:
        df[col] = df[col].astype("category")

# Verificar los tipos de datos
#print("Tipos de datos después de la conversión:")
#print(df.dtypes)

# Definir las features y la variable objetivo.
# Se eliminan las columnas que no se usarán como predictores.
X = df.drop(["gsitalta", "ds_vivo_alta", "ds_estancia", "ds_post_oper", "ds_pre_oper"], axis=1)
y = df["ds_pre_oper"]

# Identificar columnas numéricas y categóricas (se incluyen 'object' y 'category')
num_cols = X.select_dtypes(include=[np.number]).columns
cat_cols = X.select_dtypes(include=["object", "category"]).columns
#print("\nVariables categóricas que se utilizarán:", list(cat_cols))

# Crear un preprocesador que escala las variables numéricas y aplica OneHotEncoder a las categóricas
preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

# Definir un pipeline que incluya el preprocesamiento y el modelo XGBoost
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", XGBRegressor(random_state=42, objective="reg:squarederror"))
])

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir una búsqueda de hiperparámetros para XGBoost
param_grid = {
    "regressor__n_estimators": [100, 200, 300],
    "regressor__max_depth": [3, 5, 7],
    "regressor__learning_rate": [0.01, 0.1, 0.2],
    "regressor__subsample": [0.8, 1.0]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
grid_search.fit(X_train, y_train)

#print("\nMejores parámetros encontrados:", grid_search.best_params_)

# Realizar predicciones y evaluar el modelo
y_pred = grid_search.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
explained_var = explained_variance_score(y_test, y_pred)

print("\nError cuadrático medio (MSE):", mse)
print("Error absoluto medio (MAE):", mae)
print("R² (coeficiente de determinación):", r2)
print("Varianza explicada:", explained_var)

# Verificar las columnas generadas tras el preprocesamiento
preprocessor_final = grid_search.best_estimator_.named_steps["preprocessor"]

# Obtener los nombres de las columnas resultantes (requiere scikit-learn >= 1.0)
try:
    feature_names = preprocessor_final.get_feature_names_out()
    #print("\nCaracterísticas utilizadas por el modelo:")
    #print(feature_names)
except AttributeError:
    print("\nLa versión de scikit-learn no soporta 'get_feature_names_out()'.")



Error cuadrático medio (MSE): 5.568604070056622
Error absoluto medio (MAE): 1.5335294818351282
R² (coeficiente de determinación): 0.022383451461791992
Varianza explicada: 0.028431014324293402


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

# Cargar los datos
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_2/DATOS_PREPROCESADOS_2.csv')

# Eliminar filas con "-999" o -999
df = df[~df.isin(["-999", -999]).any(axis=1)]
df.drop("gidenpac", axis=1, inplace=True)

# Lista de variables categóricas a convertir
cat_features = [
    'gsitalta', 'itipsexo', 'itipingr', 'ireingre', 'iotrocen', 'gdiagalt',
    'ds_izq_der', 'ds_turno', 'ds_dia_semana_llegada', 'ds_mes_llegada',
    'ds_centro_afueras', 'ds_alergia_medicamentosa', 
    'ds_alergia_alimenticia', 'ds_otras_alergias', 'ds_ITU', 'ds_anemia',
    'ds_vitamina_d', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca',
    'ds_deterioro_cognitivo', 'ds_insuficiencia_renal', 'ds_HTA', 'ds_diabetes'
]

# Convertir estas variables a tipo 'category'
for col in cat_features:
    if col in df.columns:
        df[col] = df[col].astype("category")

# Definir X e y. Se eliminan columnas que no se usarán como predictores
X = df.drop(["gsitalta", "ds_vivo_alta", "ds_estancia", "ds_post_oper", "ds_pre_oper"], axis=1)
y = df["ds_pre_oper"]

# Identificar columnas numéricas y categóricas
num_cols = X.select_dtypes(include=[np.number]).columns
cat_cols = X.select_dtypes(include=["object", "category"]).columns

#print("Variables categóricas que se utilizarán:", list(cat_cols))

# Definir un preprocesador que escale las numéricas y codifique las categóricas
preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir una lista de modelos a comparar
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(random_state=42),
    "Lasso": Lasso(random_state=42),
    "RandomForest": RandomForestRegressor(random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, objective="reg:squarederror")
}

results = []

for name, model in models.items():
    # Crear un pipeline para cada modelo
    pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("regressor", model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    ev = explained_variance_score(y_test, y_pred)
    
    results.append({
        "Model": name,
        "MSE": mse,
        "MAE": mae,
        "R2": r2,
        "Explained Variance": ev
    })
    
    print(f"\n--- {name} ---")
    print("MSE:", mse)
    print("MAE:", mae)
    print("R²:", r2)
    print("Varianza explicada:", ev)

# Convertir los resultados a DataFrame para compararlos fácilmente
results_df = pd.DataFrame(results)
print("\nComparación de modelos:")
print(results_df)

# Opcional: Si deseas ver las características finales utilizadas por el preprocesador en alguno de los pipelines,
# puedes usar get_feature_names_out() (requiere scikit-learn >= 1.0)
preprocessor_final = Pipeline(steps=[("preprocessor", preprocessor)]).fit(X_train)
try:
    feature_names = preprocessor_final.named_steps["preprocessor"].get_feature_names_out()
    #print("\nCaracterísticas utilizadas por el modelo:")
    #print(feature_names)
except AttributeError:
    print("\nLa versión de scikit-learn no soporta 'get_feature_names_out()'.")



--- LinearRegression ---
MSE: 5.71515070667583
MAE: 1.5881819751381216
R²: -0.0033441525181225895
Varianza explicada: 0.002719618156085124

--- Ridge ---
MSE: 5.692239554768118
MAE: 1.582357876428939
R²: 0.0006780984204579621
Varianza explicada: 0.006699628023449811

--- Lasso ---
MSE: 5.732101590119526
MAE: 1.5334399534748473
R²: -0.0063200267611907
Varianza explicada: -2.220446049250313e-16

--- RandomForest ---
MSE: 6.856547759975444
MAE: 1.72355985267035
R²: -0.2037262802880635
Varianza explicada: -0.1731597210200717

--- GradientBoosting ---
MSE: 5.841003992918752
MAE: 1.5869318131225363
R²: -0.02543878576716807
Varianza explicada: -0.01829449661906679

--- XGBoost ---
MSE: 7.286377687398138
MAE: 1.8014932133216226
R²: -0.2791866064071655
Varianza explicada: -0.26064458827018666

Comparación de modelos:
              Model       MSE       MAE        R2  Explained Variance
0  LinearRegression  5.715151  1.588182 -0.003344        2.719618e-03
1             Ridge  5.692240  1.582358

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso, LassoCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

# Cargamos el dataset
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/models_v2/PREPROCESADO_2/DATOS_PREPROCESADOS_2.csv')

# Eliminar filas que contengan "-999" o -999 en cualquier columna
df = df[~df.isin(["-999", -999]).any(axis=1)]

# Eliminamos la columna identificadora
df.drop("gidenpac", axis=1, inplace=True)

# Lista de variables categóricas a tratar como tales
cat_features = [
    'gsitalta', 'itipsexo', 'itipingr', 'ireingre', 'iotrocen', 'gdiagalt',
    'ds_izq_der', 'ds_turno', 'ds_dia_semana_llegada', 'ds_mes_llegada',
    'ds_centro_afueras', 'ds_alergia_medicamentosa', 'movilidad', 'riesgo_caida',
    'ds_alergia_alimenticia', 'ds_otras_alergias', 'ds_ITU', 'ds_anemia',
    'ds_vitamina_d', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca',
    'ds_deterioro_cognitivo', 'ds_insuficiencia_renal', 'ds_HTA', 'ds_diabetes'
]

# Convertir las variables de la lista a tipo 'category'
for col in cat_features:
    if col in df.columns:
        df[col] = df[col].astype("category")

# Verificar los tipos de datos
print("Tipos de datos después de la conversión:")
print(df.dtypes)

# Definir X e y.
# Se eliminan las columnas que no se utilizarán como predictores
X = df.drop(["gsitalta", "ds_vivo_alta", "ds_estancia", "ds_post_oper", "ds_pre_oper"], axis=1)
y = df["ds_pre_oper"]

# Identificar columnas numéricas y categóricas
num_cols = X.select_dtypes(include=[np.number]).columns
cat_cols = X.select_dtypes(include=["object", "category"]).columns
print("\nVariables categóricas que se utilizarán:", list(cat_cols))

# Preprocesador: uso de RobustScaler para las variables numéricas y OneHotEncoder para las categóricas
preprocessor = ColumnTransformer(transformers=[
    ("num", RobustScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

# Usaremos Lasso con validación cruzada para la selección de características.
# Esta será la base para el RFE y también el modelo final.
# Se puede optimizar el parámetro alpha.
base_estimator = LassoCV(cv=5, random_state=42)

# Definir RFE. El número de features a seleccionar se optimizará.
rfe = RFE(estimator=base_estimator, n_features_to_select=30)

# Modelo final: Usamos Lasso (puede ser el mismo que en la selección)
regressor = Lasso(random_state=42)

# Pipeline: preprocesador -> selección de características (RFE) -> regresor
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("feature_selection", rfe),
    ("regressor", regressor)
])

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el espacio de búsqueda de hiperparámetros para optimizar:
# - El número de features a seleccionar en RFE
# - El parámetro de regularización de Lasso final (alpha)
param_grid = {
    "feature_selection__n_features_to_select": [20, 30, 40, 50],
    "regressor__alpha": [0.001, 0.01, 0.1, 1, 10]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
grid_search.fit(X_train, y_train)

print("\nMejores parámetros encontrados:")
print(grid_search.best_params_)

# Predicciones y evaluación
y_pred = grid_search.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
explained_var = explained_variance_score(y_test, y_pred)

print("\nError cuadrático medio (MSE):", mse)
print("Error absoluto medio (MAE):", mae)
print("R² (coeficiente de determinación):", r2)
print("Varianza explicada:", explained_var)

# Verificar las características utilizadas por el modelo
preprocessor_final = grid_search.best_estimator_.named_steps["preprocessor"]
try:
    feature_names = preprocessor_final.get_feature_names_out()
    print("\nCaracterísticas utilizadas tras el preprocesamiento:")
    print(feature_names)
except AttributeError:
    print("\nLa versión de scikit-learn no soporta 'get_feature_names_out()'.")


Tipos de datos después de la conversión:
itipsexo                         category
itipingr                         category
ireingre                         category
gsitalta                         category
iotrocen                         category
gdiagalt                         category
ds_izq_der                       category
ds_turno                         category
ds_edad                             int64
ds_estancia                         int64
ds_pre_oper                         int64
ds_post_oper                        int64
ds_vivo_alta                        int64
ds_dia_semana_llegada            category
ds_mes_llegada                   category
ds_centro_afueras                category
ds_alergia_medicamentosa         category
ds_alergia_alimenticia           category
ds_otras_alergias                category
ds_ITU                           category
ds_anemia                        category
ds_vitamina_d                    category
ds_insuficiencia_respiratoria    ca


Mejores parámetros encontrados:
{'feature_selection__n_features_to_select': 50, 'regressor__alpha': 0.01}

Error cuadrático medio (MSE): 5.560806414975928
Error absoluto medio (MAE): 1.5400963366501699
R² (coeficiente de determinación): 0.023752323235483708
Varianza explicada: 0.02935069662767975

Características utilizadas tras el preprocesamiento:
['num__ds_edad' 'cat__itipsexo_0' 'cat__itipsexo_1' 'cat__itipingr_0'
 'cat__itipingr_1' 'cat__ireingre_0' 'cat__ireingre_1' 'cat__iotrocen_0'
 'cat__iotrocen_1' 'cat__gdiagalt_S72.001A' 'cat__gdiagalt_S72.002A'
 'cat__gdiagalt_S72.011A' 'cat__gdiagalt_S72.012A'
 'cat__gdiagalt_S72.101A' 'cat__gdiagalt_S72.102A'
 'cat__gdiagalt_S72.141A' 'cat__gdiagalt_S72.142A' 'cat__ds_izq_der_0'
 'cat__ds_izq_der_1' 'cat__ds_turno_0' 'cat__ds_turno_1' 'cat__ds_turno_2'
 'cat__ds_dia_semana_llegada_1' 'cat__ds_dia_semana_llegada_2'
 'cat__ds_dia_semana_llegada_3' 'cat__ds_dia_semana_llegada_4'
 'cat__ds_dia_semana_llegada_5' 'cat__ds_dia_semana_llegada_6